<div style="background:#222222; color:#ffffff; padding:20px">
<h1 align="center">Guided ML With The Iris Dataset</h1>

<h2 align="center" tyle="color:#01ff84" >Learning type | Activity type | Objective |</h2>
<h2 align="center">| Supervised | Multiclass classification | Identify a flower's class |</h2>


<div>

Contents:

1. Loading the data
2. Setting up supervised learning problem (selecting features)
3. Creating a first model
    - Creating train and test datasets
    - Normalizing train and test
    - Fitting and predicting
4. Evaluate the frist model predictions
5. Crossvalidation of the model
6. Creating an end to end ML pipeline
    - Train/Test Split
    - Normalize
    - Crossvalidations
    - Model
    - fitting and predicting

## Instructions with NBGrader removed

Complete the cells beginning with `# YOUR CODE HERE` and run the subsequent cells to check your code.


Contents:
1. Loading the data
2. Setting up supervised learning problem (selecting features)
3. Creating a first model
    - Creating train and test datasets
    - Normalizing train and test
    - Fitting and predicting
4. Evaluate the frist model predictions
5. Crossvalidation of the model
6. Creating an end to end ML pipeline
    - Train/Test Split
    - Normalize
    - Crossvalidations
    - Model
    - fitting and predicting

## About the dataset

[Iris](https://archive.ics.uci.edu/ml/datasets/iris) is a well-known multiclass dataset. It contains 3 classes of flowers with 50 examples each. There are a total of 4 features for each flower.

![](./classic-datasets/images/Iris-versicolor-21_1.jpg)

## Package setups

1. Run the following two cells to initalize the required libraries. 

In [671]:
#to debug package errors
import sys
sys.path
sys.executable

'/home/tom/anaconda3/envs/ml-module/bin/python'

In [672]:
# Import needed packages
# You may add or remove packages should you need them
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline

# Display plots inline and change plot resolution to retina
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# Set Seaborn aesthetic parameters to defaults
sns.set()

## Step 1: Loading the data

1. Load the iris dataset using ```datasets.load_iris()```
2. Investigate the data structure with ```.keys()```
3. Construct a dataframe from the dataset
4. Create a 'target' and a 'class' column that contains the target names and values
5. Display a random sample of the dataframe 

In [673]:
def load_data():
    #load the dataset
    #return the dataset
    return datasets.load_iris()

load_data()['data'].shape

(150, 4)

In [674]:
assert load_data()['data'].shape == (150,4)

In [675]:
dataset = load_data()
load_data().feature_names
#dataset.keys()

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [676]:
def dataset_to_pandas():
    #put the dataset into a pandas DF using the feature names as columnsç
    #rename the column name so the dont include the '(cm)'
    #add 2 columns one with the target and another with the class
    
    # First attempt was columns=new_columns, but then I remembered list comprehensions exist
    #     new_columns = [feature.strip(" (cm)") for feature in dataset.feature_names]

    # Initially the dataframe is just the sepal and petal data
    df = pd.DataFrame(dataset.data.tolist(), columns=[feature.strip(" (cm)") for feature in dataset.feature_names])
    # For some reason numpy ndarrays can't use .replace() so I put the data into a series.
    target_series = pd.Series(dataset.target)
    class_series = target_series.replace({0: "setosa", 1:"versicolor",2:"virginica"})
    # Finally add these new series to the dataframe
    df["target"] = target_series
    df["class"] = class_series
    return df

In [677]:
df = dataset_to_pandas()
assert df['sepal length'].shape == (150,)
assert df['sepal width'].shape == (150,)
assert df['petal length'].shape == (150,)
assert df['petal width'].shape == (150,)
assert df['target'].shape == (150,)
assert df['class'].shape == (150,)


### Question
Find the X and y values we're looking for. Notice that y is categorical and thus, we could **one-hot encode it** if we are looking at **class** or we can just pick **target**. In order to one hot encode we have  to re-shape `y` it using the **.get_dummies** function. 

## For the purpose of this exercise, do not use hot encoding, go only for target but think about if you have to drop it somewhere or not...

In [678]:
df_iris = dataset_to_pandas()

In [679]:
def ohe():
    df_ohe = pd.get_dummies(df, columns=["target"], prefix="plant")
    #print(df_ohe)
    return df_ohe
    
ohe()

,sepal length,sepal width,petal length,petal width,class,plant_0,plant_1,plant_2
0,5.1,3.5,1.4,0.2,setosa,1,0,0
1,4.9,3.0,1.4,0.2,setosa,1,0,0
2,4.7,3.2,1.3,0.2,setosa,1,0,0
3,4.6,3.1,1.5,0.2,setosa,1,0,0
4,5.0,3.6,1.4,0.2,setosa,1,0,0
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,0,0,1
146,6.3,2.5,5.0,1.9,virginica,0,0,1
147,6.5,3.0,5.2,2.0,virginica,0,0,1
148,6.2,3.4,5.4,2.3,virginica,0,0,1


In [680]:
ohe_data = ohe()

assert ohe_data.shape == (150,8)

## Step 2: Setting up supervised learning problem (selecting features)

Feature selection is an essential step in improving a model's perfromance. In the first version of the model we will use the **'sepal length'** and **'sepal width'** as predicting features. Later we will see the effect of adding additional features.

1. Assign the values of the 'target' to Y as a numpy array
2. Assign the remaining feature values to X as a numpy array
3. Check the shape of X and Y. Check the first few values.
    - Can we confirm our X and Y are created correctly?

In [681]:
def target_to_numpy():
    target_array = np.array(df.target)
    return target_array
def data_to_numpy():
    data_array = np.array([ df["sepal length"],df["sepal width"] ] ).transpose()
    return data_array
   
assert_this = data_to_numpy()
print(assert_this.shape)


(150, 2)


In [682]:
Y = target_to_numpy()
X = data_to_numpy()
assert isinstance(Y, np.ndarray)
assert isinstance(X, np.ndarray)
assert X.shape == (150,2)

In [683]:
#your code here
X = df_iris[['sepal length', 'sepal width']].values
print(X.shape)
X[:5]

(150, 2)


array([[5.1, 3.5],
       [4.9, 3. ],
       [4.7, 3.2],
       [4.6, 3.1],
       [5. , 3.6]])

## Step 3: Creating the first model

In lecture we learned about creating a train and test datasets, normalizing, and fitting a model. In this step we will see how to build a simple version of this.

We have to be careful when constructing our train and test datasets. First, when we create train and test datasets we have to be careful that we always have the same datapoints in each set. Otherwise our results won't be reproduceable or we might introduce a bias into our model.

We also need to be attentive to when we normalize the data. What would be the effect of normalizing the data (i.e. with StandardScaler to a range between 0 - 1) before we create our train and test sets? Effectively we would use information in the test set to structure the values in the training set and vice versa. Therefore normalizing train and test independently is the preferred method.

1. Create X_train, X_test, Y_train, Y_test using ```train_test_split()``` with an 80/20 train/test split. Look in the SKLearn documentation to understand how the function works.
    - Inspect the first few rows of X_train.
    - Run the cell a few times. Do the first few rows change?
    - What option can we use in ```train_test_split()``` to stop this from happening?
2. Normalize the train and test datasets with ```StandardScaler```
    - We can fit the transform with ```.fit()``` and ```.transform()``` to apply it. Look in the documentation for an esample of how to do this.
    - Does it make sense to normalize Y_train and Y_test?
3. Initalize a ```LogisticRegression()``` model and use the ```.fit()``` method to initalize the first model.
    - We will pass the X_train and Y_train variables to the ```.fit()``` method.
    - Once the model is fit, use the ```.predict()``` with the X_test and save the output as predictions.

In [701]:
#split train and test data 80/20
# X is the data, Y is the target - I'm just going to re-use the X and Y above
#X_train, X_test, Y_train, Y_test = train_test_split(data_to_numpy(),target_to_numpy(),train_size=0.8)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8, random_state=22)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(120, 2)
(120,)
(30, 2)
(30,)


In [685]:
assert X_train.shape == (120,2)
assert Y_train.shape == (120,)
assert X_test.shape  == (30,2)
assert Y_test.shape  == (30,)

In [686]:
#create and fit the scaler object on the training data
scaler = StandardScaler()
# Fit the model to the training data
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train[:5]

array([[-1.08230057, -1.66328924],
       [ 0.49268906, -0.76724117],
       [ 0.97730126,  0.57683095],
       [-1.08230057,  0.57683095],
       [-0.96114752,  1.02485499]])

In [687]:
assert np.amin(X_train) >= -2.5
assert np.amax(X_train) <= 3.2
assert np.amin(X_test) >= -2
assert np.amin(X_test) <= 2.75

In [688]:
# Make predictions using logistic regression, on the training data
clf = LogisticRegression(solver="lbfgs",multi_class="multinomial").fit(X_train, Y_train)
# Score using the test data

predicitions = clf.predict(X_test)
score = clf.score(X_test, Y_test)
print("Here are some predictions...")
print(predicitions)
print("These predictions are {}% accurate!".format(round(score*100,2)))

Here are some predictions...
[1 2 2 0 0 1 0 1 1 2 2 1 0 0 2 1 0 0 0 0 0 2 1 0 2 2 0 0 1 2]
These predictions are 83.33% accurate!


In [689]:
assert predictions.shape == (30,)

## Step 4: Evaluate the frist model's predictions

We will learn more about how to evaluate the performance of a classifier in later lessons. For now we will use % accuracy as our metric. It is important to know that this metric only helps us understand the specific performance of our model and not, for example, where we can improve it, or where it already perfoms well.

1. Use ```.score()``` to evaluate the performance of our first model.

In [690]:
score = 0
#evaluating the performace of our first model
#your code here
score = clf.score(X_train, Y_train)
print(score)

0.8166666666666667


In [691]:
assert score >=0.7

In [692]:
assert score >=0.72

In [693]:
assert score >=0.73

## Step 5: Crossvalidation of the model
Our first model achived ~90% accruacy. This is quite good. How do we know it is reproducable? If we run the model again and our performance is 85% which is correct? And what about improving our model? Can you think of one thing we can do to potentially improve the model?

#### Crossvalidation
Corssvalidation is when we create multiple X and Y datasets. On each dataset we train and fit the model. We then average the results and return a 'crossvalidated' accruacy.

1. Initalize a new version of the model you trained above with the same paramters.
2. Use ```cross_validate()``` to run the model with 5 crossvalidation folds. 

In [702]:
#model with cross validation
#your code here

#cross validate the training set
clf_cv = 0
CV = 0
cv = cross_validate(clf, X_train, Y_train)

def print_scores(cv):
    #print out cross validation scores
    print("-----")
    print("CROSS VALIDATION")
    [print('Crossvalidation fold: {}  Accruacy: {}'.format(n, score)) for n, score in enumerate(cv['test_score'])]
    #print out the mean of the cross validation
    print('Mean train cross validation score {}'.format(cv['test_score'].mean()))

print_scores(cv)

-----
CROSS VALIDATION
Crossvalidation fold: 0  Accruacy: 0.8333333333333334
Crossvalidation fold: 1  Accruacy: 0.75
Crossvalidation fold: 2  Accruacy: 0.9166666666666666
Crossvalidation fold: 3  Accruacy: 0.7916666666666666
Crossvalidation fold: 4  Accruacy: 0.7916666666666666
Mean train cross validation score 0.8166666666666667


In [703]:
assert len(cv['test_score']) == 5
assert max(cv['test_score']) >= 0.85
assert min(cv['test_score']) >= 0.69
assert cv['test_score'].mean() >= 0.77

## Step 6: Creating an end to end ML pipeline
Congraulations you've trained, crossvalidated, predicted, and evaluated your frist classifier. Now that you understand the basic steps we will look at a way to combine all these steps together.

Before we go further think about what you would have to do if you wanted to change the model. Intalize a new model, change the vairables, redo the cross validation...etc. Seems like a lot. And when we have to change lots of code it is easy to make mistakes. And what if you wanted to try many models and see which one performed best? Or try changing many different features? How could you do it without writing each one out as we have?

The solution is to use SKLearn's pipeline class. A pipeline is an object that will execute the various steps in the machine learning process. We can choose what elements we want in the pipeline and those that we do not. Once setup, we can rapidly change models, or input data and have it return our results in an ordered way.


1. Initalize a scaler and a classifer object like we did previously.
2. Use the ```make_pipeline()``` function to construct a transofmraiton pipeline for the scaler and the classifier
3. Input the pipeline object to the cross_validator and evaluate with 5 folds.
4. Print out your results (hint: make a function for repetitve tasks like printing)

In [708]:
#define the scaler
#define the classifier
#make the pipeline
#run the cross validation
#print results
scaler = scaler = StandardScaler()
classifier = LogisticRegression(solver="lbfgs",multi_class="multinomial").fit(X_train, Y_train)
pipe = make_pipeline(scaler, classifier)
scores = 0
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8, random_state=10)
#normalize the dataset
preprocessing.normalize(X_train, copy=False)
#create and fit the scaler object on the training data
scaler = StandardScaler()
scaler.fit(X_train)
# scaler.transform(X_train)
#scaler.fit_transform(X_train, Y_train)
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
predictions = log_reg.predict(X_test)
log_reg.score(X_test, Y_test)

def make_pipeline(scaler, log_reg):
    cv = cross_validate(log_reg, X_test, Y_test)


def print_scores(cv):
    #print out cross validation scores
    [print('Crossvalidation fold: {}  Accruacy: {}'.format(n, score)) for n, score in enumerate(cv['test_score'])]
    #print out the mean of the cross validation
    print('Mean train cross validation score {}'.format(cv['test_score'].mean()))

print_scores(cv)
    

Crossvalidation fold: 0  Accruacy: 0.8333333333333334
Crossvalidation fold: 1  Accruacy: 0.75
Crossvalidation fold: 2  Accruacy: 0.9166666666666666
Crossvalidation fold: 3  Accruacy: 0.7916666666666666
Crossvalidation fold: 4  Accruacy: 0.7916666666666666
Mean train cross validation score 0.8166666666666667


In [709]:
assert type(pipe) == type(make_pipeline(scaler, classifier))
assert len(cv['test_score']) == 5
assert max(cv['test_score']) >= 0.83
assert min(cv['test_score']) >= 0.69
assert cv['test_score'].mean() >= 0.74

## Challenge Exercise

In this notebook we only used two features to predict the class of the flower. We also did not do any hypter parameter tuning. The challenge is to impove the prediction results. Some ideas we can try:
1. Add features to the input and run the cross validation pipeline
2. Investigate how to use ```GridSearchCV```, a powerful funtion that searches through hyperparmetrs and does cross validation.
    - Hint: Input the pipeline directly into GridSearchCV
3. Try a different models like RandomForest or SVM.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()